In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

from PIL import Image

In [ ]:
train = pd.read_csv('./train.csv')

In [ ]:
train['Category'].unique()

array(['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts',
       'Women Tops & Tunics'], dtype=object)

In [5]:
# Sarees , Women Tshirts , Women Tops & Tunics
category = ['Sarees']

In [ ]:
# For Kaggle

train['image_path'] = train.apply(lambda x: './Data/train_images/' + x['image_path'].split('/')[-1], axis=1)

In [7]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
])

In [8]:
def prepare_images(image_paths, image_size=(224, 224)):
    """
    Converts image paths into augmented tensors using predefined data augmentation layers.
    
    Args:
        image_paths (list of str): List of image paths.
        image_size (tuple): Target size of images (height, width).
    
    Returns:
        np.array: Array of preprocessed and augmented image tensors.
    """

    images = []
    
    for path in image_paths:
        try:
            img = Image.open(path).convert("RGB")
            img = img.resize(image_size)  
            img_array = np.array(img) / 255.0  
            
            # Add batch dimension and convert to tensor
            img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)
            img_tensor = tf.expand_dims(img_tensor, axis=0)  
            
            # Apply data augmentation
            augmented_img = data_augmentation(img_tensor)[0].numpy()  
            images.append(augmented_img)

        except Exception as e:
            print(f"Error loading image {path}: {e}")
    
    return np.array(images)


In [ ]:
def predict_labels(unique_labels, x_train, y_train, x_predict):
    """
    Train a ResNet50-based classifier and predict labels.
    
    Args:
        unique_labels (int): Number of unique labels in the dataset.
        x_train (np.array): Preprocessed training images.
        y_train (np.array): Labels for training images.
        x_predict (np.array): Preprocessed images to predict.
    
    Returns:
        list of dict: List of dictionaries with structure {'id': <id>, 'label': <label>}.
    """
    # Convert labels to categorical
    y_train_categorical = to_categorical(y_train, num_classes=unique_labels)
    
    # Load ResNet50 with pretrained weights
    base_model = ResNet50(weights='./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model
    
    # Build a simple classification model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(unique_labels, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(x_train, y_train_categorical, epochs=4, batch_size=32, verbose=1)
    
    # Predict labels for x_predict
    predictions = model.predict(x_predict)
    y_predict = np.argmax(predictions, axis=1)
    
    return y_predict


In [ ]:
for cat in category:
    df = train[train['Category'] == cat]
    
    df.dropna(axis=1, how='all' , inplace=True)

    labelcolumns = [ 'attr_8', 'attr_9', 'attr_10']
    # for col in df.columns: 
    #     if col.startswith('attr'): labelcolumns.append(col) 

    for i in labelcolumns:
        predict_df = df[df[i].isna()][['id' , 'image_path']]
        train_df = df[~df[i].isna()][['id' , 'image_path' , i]]
        unique_val = len(train_df[i].unique())

        le = LabelEncoder()

        train_df[i] = le.fit_transform(train_df[i])

        class_to_label = dict(zip(le.classes_, range(len(le.classes_))))
        label_to_class = {v:k  for k, v in class_to_label.items()}

        y_train = train_df[i]

        x_train = prepare_images(train_df['image_path'])
        x_predict = prepare_images(predict_df['image_path'])

        # Step 2: Predict labels
        y_predict = predict_labels(
            unique_val,
            x_train,
            y_train,
            x_predict
        )
        
        # Step 3: Prepare final result
        predict_df['label'] = [label_to_class[label] for label in y_predict]
        result = predict_df[['id', 'label']]

        result.to_csv(f'./output/{cat}/{i}.csv', index=False)


['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']


C:\Users\Anubhav Vashishtha\AppData\Local\Temp\ipykernel_13972\470146512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all' , inplace=True)


In [69]:
print('done')

done
